# Imports

In [100]:
import math
import numpy as np

# Solution of past year

We solve this problem:

$( p_l, p_h) \mapsto
(320 - p_l) \int_{160}^{p_l} (\alpha r + \beta) \, dr
+ (320 - p_h) \int_{p_l}^{p_h} (\alpha r + \beta) \, dr
\left(
    \mathbb{1}_{\bar{p} \leq p_h} +
    \mathbb{1}_{\bar{p} > p_h} \cdot
    \frac{320 - \bar{p}}{320 - p_h}
\right). $


Note that:

$\int_{a}^{b} (\alpha r + \beta)\,dr
= -\frac{\alpha}{2}\bigl(b^2 - a^2\bigr) + \beta\,(b - a). $

In [101]:
def solve(p_avg):
    """
    Given the average of second bids, find the optimal low and high bids,
    using an integration interval that starts at 160 instead of 260.

    Parameters
    ----------
    p_avg : float
        Average value of second bids.

    Returns
    -------
    argmax : list of tuple
        Maximizers.
    val_max : float
        Maximal profit.
    """
    val_max = float('-inf')
    argmax = []


    # (You may also wish to adjust these bounds for your particular problem.)
    for l in range(160, 320):
        for h in range(l, 320):
            temp = (320 - l) * (-(4/103) * (-160 + l) + (-12800 + l**2/2)) / 5150
            temp2 = (320 - h) * (-(4/103) * (h - l) + (h**2/2 - l**2/2)) / 5150
            val = temp + temp2 * (1 if p_avg <= h else (320-p_avg)/(320-h))

            if math.isclose(val, val_max):
                argmax.append((l, h, val))
            if val > val_max:
                val_max = val
                argmax = [(l, h)]
    return argmax, val_max

In [102]:
for p_avg in np.arange(250,320,0.5):
    argmax, val_max = solve(p_avg)
    if len(argmax) > 1:
        print("p_avg:", p_avg, "  Maximizers:", argmax, " Profit:", f"{val_max:.2f}")
    else:
        print("p_avg:", p_avg, "  Maximizer:", argmax[0], " Profit:", f"{val_max:.2f}")

p_avg: 250.0   Maximizer: (221, 273)  Profit: 340.54
p_avg: 250.5   Maximizer: (221, 273)  Profit: 340.54
p_avg: 251.0   Maximizer: (221, 273)  Profit: 340.54
p_avg: 251.5   Maximizer: (221, 273)  Profit: 340.54
p_avg: 252.0   Maximizer: (221, 273)  Profit: 340.54
p_avg: 252.5   Maximizer: (221, 273)  Profit: 340.54
p_avg: 253.0   Maximizer: (221, 273)  Profit: 340.54
p_avg: 253.5   Maximizer: (221, 273)  Profit: 340.54
p_avg: 254.0   Maximizer: (221, 273)  Profit: 340.54
p_avg: 254.5   Maximizer: (221, 273)  Profit: 340.54
p_avg: 255.0   Maximizer: (221, 273)  Profit: 340.54
p_avg: 255.5   Maximizer: (221, 273)  Profit: 340.54
p_avg: 256.0   Maximizer: (221, 273)  Profit: 340.54
p_avg: 256.5   Maximizer: (221, 273)  Profit: 340.54
p_avg: 257.0   Maximizer: (221, 273)  Profit: 340.54
p_avg: 257.5   Maximizer: (221, 273)  Profit: 340.54
p_avg: 258.0   Maximizer: (221, 273)  Profit: 340.54
p_avg: 258.5   Maximizer: (221, 273)  Profit: 340.54
p_avg: 259.0   Maximizer: (221, 273)  Profit: 

# Simulation on expected profits computed from uniform distribution

In [104]:
# -----------------------------------------------------------
# Function: sample_reserve
# -----------------------------------------------------------
def sample_reserve(r_config, size):
    """
    Samples Turtle reserve prices from a disjoint, piecewise uniform distribution.

    Parameters:
      r_config : tuple
          Defines the two intervals for the reserve price.
          Example: (160, 200, 250, 320) means reserve is uniformly drawn
          from [160,200] or from [250,320].
      size : int
          Number of random samples to generate.

    Returns:
      samples : np.array
          Array of reserve price samples.
    """
    # Unpack the intervals
    r1_low, r1_high, r2_low, r2_high = r_config
    # Compute the lengths of the two intervals
    L1 = r1_high - r1_low
    L2 = r2_high - r2_low
    total_length = L1 + L2
    # Decide for each sample whether to draw from interval 1 or 2
    u = np.random.uniform(0, 1, size)
    samples = np.empty(size)
    idx1 = u < (L1 / total_length)  # proportion from the first interval
    n1 = np.sum(idx1)
    n2 = size - n1
    samples[idx1] = np.random.uniform(r1_low, r1_high, n1)
    samples[~idx1] = np.random.uniform(r2_low, r2_high, n2)
    return samples

# -----------------------------------------------------------
# Function: simulate_profit
# -----------------------------------------------------------
def simulate_profit(b1, b2, A, r_config, selling_price=320, N=10000):
    """
    Simulates the average profit for given bid levels (b1 and b2) and
    average competitor bid A, under the new rule:

      - Round 1:
          If the Turtle's reserve r ≤ b1, it accepts
          and profit = selling_price - b1.
      - Round 2:
          * If b2 > A and r < b2, profit = selling_price - b2.
          * If b2 < A and r < b2, profit = (selling_price - b2) * p,
            where p = ((selling_price - A)**3)/(selling_price - b2).
          (No profit if r ≥ b2.)

    Parameters
    ----------
    b1 : float
        First-round bid.
    b2 : float
        Second-round bid (should satisfy b2 >= b1).
    A : float
        Average competitor second bid.
    r_config : tuple
        Defines intervals for the reserve price distribution, e.g. (160,200,250,320).
    selling_price : float, optional
        Price at which you resell the item (default = 320).
    N : int, optional
        Number of Monte Carlo samples (default = 10000).

    Returns
    -------
    avg_profit : float
        Average profit over N simulated draws from the piecewise reserve distribution.
    """
    # 1) Draw N Turtle reserve prices from disjoint intervals
    reserves = sample_reserve(r_config, N)

    # 2) Initialize profit array
    profits = np.zeros(N)

    # 3) Round 1: If reserve ≤ b1, you get the Flipper for b1
    round1_success = (reserves <= b1)
    profits[round1_success] = selling_price - b1

    # 4) Round 2: Only for Turtles that didn't accept round 1
    round1_failure = ~round1_success

    # Case A: b2 > A → normal acceptance if r < b2
    round2_success_above = round1_failure & (b2 > A) & (reserves < b2)
    profits[round2_success_above] = selling_price - b2

    # Case B: b2 < A → scaled acceptance if r < b2
    round2_success_below = round1_failure & (b2 < A) & (reserves < b2)
    if b2 < selling_price:  # To avoid any division by zero if b2 == 320
        scale_factor = ((selling_price - A) / (selling_price - b2))**3
        profits[round2_success_below] = (selling_price - b2) * scale_factor

    # 5) Average profit over all samples
    return np.mean(profits)

# -----------------------------------------------------------
# Function: optimize_strategy
# -----------------------------------------------------------
def optimize_strategy(A, r_config, selling_price=320,
                      b1_min=160, b1_max=320, b2_min=160, b2_max=320,
                      step=1, N=10000):
    """
    Searches over first and second bid pairs (b1 and b2) to find the pair
    that maximizes the average profit, given an average competitor bid A and
    a Turtle reserve distribution defined by r_config.

    Parameters:
      A : int or float
          The average competitor second bid.
      r_config : tuple
          Reserve price configuration (e.g., (160,200,250,320)).
      selling_price : int or float, default=320
      b1_min, b1_max : int
          The allowable range for the first bid.
      b2_min, b2_max : int
          The allowable range for the second bid.
      step : int
          The discretization step for the grid search.
      N : int
          Number of Monte Carlo samples for each bid pair evaluation.

    Returns:
      best_b1, best_b2 : optimal first and second bid values found.
      best_profit : The corresponding average simulated profit.
    """
    best_profit = -np.inf
    best_b1, best_b2 = None, None

    # Loop over all candidate bids for first and second rounds.
    for b1 in range(b1_min, b1_max + 1, step):
        for b2 in range(b1, b2_max + 1, step):  # Enforce b2 >= b1.
            profit = simulate_profit(b1, b2, A, r_config, selling_price, N)
            if profit > best_profit:
                best_profit = profit
                best_b1, best_b2 = b1, b2

    return best_b1, best_b2, best_profit

# -----------------------------------------------------------
# Main simulation: Varying A and reserve parameters (r_config)
# -----------------------------------------------------------
if __name__ == '__main__':
    # Reserve distribution configurations can be stored in a dictionary.
    r_configs = {
        "config1": (160, 200, 250, 320),    # Original configuration.
    }

    # Define a range of average competitor second bids A.
    A_values = np.arange(160, 321, 5)  # For example, from 160 to 320 in steps of 20.

    # Dictionary to store results for each configuration.
    results = {}

    for config_name, r_config in r_configs.items():
        config_results = []
        print(f"\n--- Results for {config_name} ---")
        for A in A_values:
            b1_opt, b2_opt, profit_opt = optimize_strategy(A, r_config, N=5000, step=1)
            config_results.append((A, b1_opt, b2_opt, profit_opt))
            print(f"Average competitor bid A={A:3d} => Optimal b1={b1_opt:3d}, b2={b2_opt:3d} with profit={profit_opt:7.2f}")
        results[config_name] = config_results



--- Results for config1 ---
Average competitor bid A=160 => Optimal b1=200, b2=285 with profit=  55.62
Average competitor bid A=165 => Optimal b1=198, b2=284 with profit=  56.26
Average competitor bid A=170 => Optimal b1=200, b2=283 with profit=  55.73
Average competitor bid A=175 => Optimal b1=200, b2=286 with profit=  55.94
Average competitor bid A=180 => Optimal b1=201, b2=280 with profit=  55.68
Average competitor bid A=185 => Optimal b1=201, b2=287 with profit=  55.88
Average competitor bid A=190 => Optimal b1=200, b2=290 with profit=  55.95
Average competitor bid A=195 => Optimal b1=200, b2=291 with profit=  55.69
Average competitor bid A=200 => Optimal b1=200, b2=281 with profit=  56.15
Average competitor bid A=205 => Optimal b1=200, b2=282 with profit=  56.04
Average competitor bid A=210 => Optimal b1=200, b2=277 with profit=  56.38
Average competitor bid A=215 => Optimal b1=200, b2=279 with profit=  56.31
Average competitor bid A=220 => Optimal b1=199, b2=281 with profit=  56